# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770841077835         NaN   2.98e-01   0.80    8.0
  2   -2.772140518557   -1.30e-03   4.86e-02   0.80    1.0
  3   -2.772170032052   -2.95e-05   2.80e-03   0.80    1.0
  4   -2.772170706597   -6.75e-07   2.97e-04   0.80    2.0
  5   -2.772170720765   -1.42e-08   1.33e-04   0.80    2.0
  6   -2.772170722967   -2.20e-09   1.57e-05   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770747794631         NaN   2.99e-01   0.80    9.0
  2   -2.772059354448   -1.31e-03   4.77e-02   0.80    1.0
  3   -2.772082688343   -2.33e-05   4.19e-03   0.80    2.0
  4   -2.772083417548   -7.29e-07   9.75e-05   0.80    2.0
  5   -2.772083417803   -2.55e-10   1.34e-05   0.80    2.0


1.7736253340288606

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770796727815         NaN   2.99e-01   0.80    8.0
  2   -2.772055088156   -1.26e-03   4.81e-02   0.80    1.0
  3   -2.772082886696   -2.78e-05   3.03e-03   0.80    1.0
  4   -2.772083378914   -4.92e-07   5.01e-04   0.80    2.0
  5   -2.772083416021   -3.71e-08   1.36e-04   0.80    2.0
  6   -2.772083417787   -1.77e-09   9.95e-06   0.80    2.0

Polarizability via ForwardDiff:       1.7725523061238613
Polarizability via finite difference: 1.7736253340288606
